In [1]:
import numpy as np, pandas as pd
import train as t
from src import helper_models as hm, helper_data as hd, helper_tts as htts
from importlib import reload
import keras, glob




Using TensorFlow backend.
/mnt/volume2Tb/Dropbox/PROJECTS/MACHINE-LEARNING/AQUASCOPE/plankifier/src/helper_models.py:235: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(self.modelkind!='feat', self.modelkind!='mixed', "We only augment with image data")
/mnt/volume2Tb/Dropbox/PROJECTS/MACHINE-LEARNING/AQUASCOPE/plankifier/src/helper_models.py:530: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(input_shape[0]) == 3, "MixedModel: first dimension should be images, but has shape {}".format(input_shape[0]))
/mnt/volume2Tb/Dropbox/PROJECTS/MACHINE-LEARNING/AQUASCOPE/plankifier/src/helper_models.py:531: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(input_shape[1]) == 1, "MixedModel: second dimension should be features, but has shape {}".format(input_shape[1]))


In [2]:
# Reload imports of custom modules, in case I am modifying and reloading them live
reload(t)
reload(hd)
reload(hm)
reload(htts)


<module 'src.helper_tts' from '/mnt/volume2Tb/Dropbox/PROJECTS/MACHINE-LEARNING/AQUASCOPE/plankifier/src/helper_tts.py'>

# Initialize simulation

In [3]:
sim=t.Ctrain()

No model was specified by the user, so we analyze images with conv2
kind: image


# Init parameters
Parameters must always be updated through the `UpdateParams()` method

In [4]:
def SetModelParams(kind):
    ''' 
    A quick way to set compatible user parameters of data and model kinds
    
    Input: kind = either 'image', 'feat' or 'mixed'
    Output: model_image, model_feat, datakind, ttkind, aug
    '''
    
    if kind == 'image':
        return ('conv2', None, 'image', 'image', True)
    
    elif kind == 'feat':
        return (None, 'mlp', 'feat', 'feat', False)
    
    elif kind == 'mixed':
        return ('conv2', 'mlp', 'mixed', 'mixed', False)
        

In [5]:
model_image, model_feat, datakind, ttkind, aug = SetModelParams('image')

sim.UpdateParams(
    datapaths=['./data/1_zooplankton_0p5x/training/zooplankton_trainingset_2020.04.28/',
               './data/1_zooplankton_0p5x/training/zooplankton_trainingset_2020.07.06/'],
    outpath     = 'out_example', 
    L           = 128, 
    aug         = aug, 
    model_feat  = model_feat,
    model_image = model_image,
    datakind    = datakind, 
    ttkind      = ttkind,
    class_select= ['chaoborus','bosmina','unknown_plankton'] #None
    )


Create output directory only after you've set the right `outpath`

In [6]:
sim.CreateOutDir()


In [7]:
print(sim.params.datapaths)


['./data/1_zooplankton_0p5x/training/zooplankton_trainingset_2020.04.28/', './data/1_zooplankton_0p5x/training/zooplankton_trainingset_2020.07.06/']


# Load data

In [8]:
# These arguments are the defaults
reload(hd)
sim.LoadData(L=sim.params.L, class_select=sim.params.class_select, datakind=sim.params.datakind) 


datapaths: ['./data/1_zooplankton_0p5x/training/zooplankton_trainingset_2020.04.28/', './data/1_zooplankton_0p5x/training/zooplankton_trainingset_2020.07.06/']
classes from datapaths: ['diaphanosoma', 'synchaeta', 'keratella_cochlearis', 'paradileptus', 'rotifers', 'nauplius', 'asplanchna', 'hydra', 'daphnia_skins', 'dirt', 'leptodora', 'unknown_plankton', 'trichocerca', 'polyarthra', 'diatom_chain', 'maybe_cyano', 'copepod_skins', 'bosmina', 'fish', 'cyclops', 'asterionella', 'chaoborus', 'daphnia', 'eudiaptomus', 'uroglena', 'keratella_quadrata', 'fragilaria', 'ceratium', 'dinobryon', 'filament', 'kellikottia', 'unknown', 'aphanizomenon', 'conochilus']
class: chaoborus (10)
class: bosmina (81)
class: unknown_plankton (71)


# Create sets (test and train)

In [9]:
sim.UpdateParams(testSplit=0.25)
sim.CreateTrainTestSets(ttkind=sim.params.ttkind)


# Create and Train Model
Custom parameter changes are made by acting directly on the params class through the `UpdateParams()` method.

In [10]:
sim.UpdateParams(totEpochs=10)
sim.Train()

modelkind: image
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Epoch 1/10
3/3 [==============================] - 3s 892ms/step - loss: 0.9672 - accuracy: 0.4896 - val_loss: 1.0639 - val_accuracy: 0.5854
Epoch 2/10
3/3 [==============================] - 2s 549ms/step - loss: 0.9239 - accuracy: 0.4878 - val_loss: 1.0604 - val_accuracy: 0.4390
Epoch 3/10
3/3 [==============================] - 0s 107ms/step - loss: 0.8961 - accuracy: 0.5506 - val_loss: 1.0550 - val_accuracy: 0.5366
Epoch 4/10
3/3 [==============================] - 0s 85ms/step - loss: 0.8464 - accuracy: 0.5521 - val_loss: 1.0532 - val_accuracy: 0.4390
Epoch 5/10
3/3 [==============================] - 0s 88ms/step - loss: 0.8769 - accuracy: 0.4944 - val_loss: 1.0512 - val_accuracy: 0.4146
Epoch 6/10
3/3 [==============================] - 0s 101ms/step - loss: 0.6687 - accuracy: 0.6742 - val_loss: 1.0441 - val_accuracy: 0.6098
Epoch 7/10
3/3 [==============================] - 0s 104ms/ste

# Classification Report

In [11]:
sim.Report()

                  precision    recall  f1-score   support

         bosmina       0.77      0.68      0.72        25
       chaoborus       0.00      0.00      0.00         2
unknown_plankton       0.47      0.64      0.55        14

        accuracy                           0.63        41
       macro avg       0.42      0.44      0.42        41
    weighted avg       0.63      0.63      0.63        41



/opt/miniconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


We now extract manually the training loss corresponding to the best weights, so that we can make sure that restarting the simulation works

In [12]:
def BestLoss(history):
    ''' Returns the training loss of the point where the validation loss was minimal'''
    return history['loss'][np.argmin(history['val_loss'])]
def InitLoss(history):
    ''' Returns the training loss of the point where the validation loss was minimal'''
    return history['loss'][0]

# initLoss = InitLoss(sim.history.history)
# bestLoss = BestLoss(sim.history.history)

print('Initial training loss:',InitLoss(sim.history.history))
print('Best    training loss:',BestLoss(sim.history.history))


Initial training loss: 0.9672347903251648
Best    training loss: 0.805130074533184


# Start again from scratch
If we train again, the simulation does not start again where it ended, but it starts from scratch.
Since the default initialization is random, the initial value will be close to that of the previous run, but not the same.

In [13]:
sim.Train()

modelkind: image
Epoch 1/10
3/3 [==============================] - 0s 136ms/step - loss: 1.1948 - accuracy: 0.4583 - val_loss: 1.0766 - val_accuracy: 0.6341
Epoch 2/10
3/3 [==============================] - 0s 99ms/step - loss: 1.0267 - accuracy: 0.4494 - val_loss: 1.0716 - val_accuracy: 0.6341
Epoch 3/10
3/3 [==============================] - 0s 104ms/step - loss: 0.9553 - accuracy: 0.4944 - val_loss: 1.0647 - val_accuracy: 0.6098
Epoch 4/10
3/3 [==============================] - 0s 84ms/step - loss: 0.8497 - accuracy: 0.6742 - val_loss: 1.0585 - val_accuracy: 0.6098
Epoch 5/10
3/3 [==============================] - 0s 87ms/step - loss: 0.8645 - accuracy: 0.4896 - val_loss: 1.0562 - val_accuracy: 0.6098
Epoch 6/10
3/3 [==============================] - 0s 92ms/step - loss: 0.8792 - accuracy: 0.5976 - val_loss: 1.0496 - val_accuracy: 0.6585
Epoch 7/10
3/3 [==============================] - 0s 110ms/step - loss: 0.8100 - accuracy: 0.5417 - val_loss: 1.0482 - val_accuracy: 0.5854
Epoch 8

In [14]:
print('Initial training loss:',InitLoss(sim.history.history))
print('Best    training loss:',BestLoss(sim.history.history))


Initial training loss: 1.1947825749715169
Best    training loss: 0.6574848207195153


# Start from a previous state
We have two ways of not starting from scratch. We can either *(a)* define the full model and load the weight configuration, or *(b)* load the full model. We could also load the full model and then load a different weight configuration *(b+a)*.

I also show how to play around with some input parameters.



## (a) Define model and load weights from disk
We don't update the parameters, because we keep the same ones as in the previous runs, for comparison. The only thing we need to do, before calling `sim.Train()`, is to specify the file with the weight configuration through the `load_weights` parameter.

We will load the weights that minimized the test loss in the previous run.
You will see now that the initial loss is lower value than that of the two previous runs.

In [15]:
# Load the weights
sim.params.load_weights=sim.params.outpath+'/bestweights.hdf5'
sim.Train()

modelkind: image
Loading weights from  out_example/bestweights.hdf5
Epoch 1/10
3/3 [==============================] - 0s 128ms/step - loss: 0.6639 - accuracy: 0.7191 - val_loss: 1.0327 - val_accuracy: 0.6341
Epoch 2/10
3/3 [==============================] - 0s 90ms/step - loss: 0.6586 - accuracy: 0.7191 - val_loss: 1.0297 - val_accuracy: 0.6829
Epoch 3/10
3/3 [==============================] - 0s 106ms/step - loss: 0.7776 - accuracy: 0.6875 - val_loss: 1.0259 - val_accuracy: 0.6341
Epoch 4/10
3/3 [==============================] - 0s 86ms/step - loss: 0.7771 - accuracy: 0.5506 - val_loss: 1.0249 - val_accuracy: 0.6341
Epoch 5/10
3/3 [==============================] - 0s 88ms/step - loss: 0.7337 - accuracy: 0.6292 - val_loss: 1.0232 - val_accuracy: 0.6585
Epoch 6/10
3/3 [==============================] - 0s 93ms/step - loss: 0.7900 - accuracy: 0.5618 - val_loss: 1.0198 - val_accuracy: 0.6341
Epoch 7/10
3/3 [==============================] - 0s 104ms/step - loss: 0.6485 - accuracy: 0.708

In [16]:
print('Initial training loss:', InitLoss(sim.history.history))
print('Best    training loss:', BestLoss(sim.history.history))


Initial training loss: 0.6626426584265205
Best    training loss: 0.6060905610577444


## (b) Loading full model from disk (CHECKPOINTING)
This is essentially what we do when checkpointing. At the end of each run, the entire model is saved (the default name is `'keras_model.h5'`). If we want to restart from there, we just need to load that model.

Since in this case we are checkpointing, we also show how to handle the simulation times.
Since when checkpointing one usually is starting a simulation from scratch, we will **define a new Ctrain class that loads all the parameters**.


In [17]:
# Load model, classes and parameters from the end of the previous run
model_from_previous  = sim.params.outpath+'/'+sim.params.saveModelName
params_from_previous = np.load(sim.params.outpath+'/params.npy', allow_pickle=True).item()
classes_from_previous = np.load(sim.params.outpath+'/classes.npy')

In [18]:
# Create a new class, just as if we were starting a new simulation that loaded a previous checkpoint
sim2=t.Ctrain()
sim2.params = params_from_previous
sim2.CreateOutDir()
sim2.LoadData(L=sim.params.L, class_select=sim.params.class_select, datakind=sim.params.datakind) # Should make a deep copy from sim
sim2.CreateTrainTestSets(ttkind=sim.params.ttkind)

No model was specified by the user, so we analyze images with conv2
kind: image
datapaths: ['./data/1_zooplankton_0p5x/training/zooplankton_trainingset_2020.04.28/', './data/1_zooplankton_0p5x/training/zooplankton_trainingset_2020.07.06/']
classes from datapaths: ['diaphanosoma', 'synchaeta', 'keratella_cochlearis', 'paradileptus', 'rotifers', 'nauplius', 'asplanchna', 'hydra', 'daphnia_skins', 'dirt', 'leptodora', 'unknown_plankton', 'trichocerca', 'polyarthra', 'diatom_chain', 'maybe_cyano', 'copepod_skins', 'bosmina', 'fish', 'cyclops', 'asterionella', 'chaoborus', 'daphnia', 'eudiaptomus', 'uroglena', 'keratella_quadrata', 'fragilaria', 'ceratium', 'dinobryon', 'filament', 'kellikottia', 'unknown', 'aphanizomenon', 'conochilus']
class: chaoborus (10)
class: bosmina (81)
class: unknown_plankton (71)


**Load model without training it, and make a classification report**. You can see that the model is loaded correctly, since it gives better than random predictions.

In [19]:
sim2.Train(train=False) #This only loads the model
sim2.Report()

modelkind: image
Training took 0.0014480988184611002 minutes
Saving the last model. These are not the best weights, they are the last ones. For the best weights use the callback output (bestweights.hdf5)]
                  precision    recall  f1-score   support

         bosmina       0.82      0.36      0.50        25
       chaoborus       0.00      0.00      0.00         2
unknown_plankton       0.40      0.71      0.51        14

        accuracy                           0.46        41
       macro avg       0.41      0.36      0.34        41
    weighted avg       0.64      0.46      0.48        41



Now let's prolongate the run for some more steps.
Since we are prolongating a run, we also want to make sure that the timesteps are consistent

In [20]:
# Set initial epoch to the end of the previous run, and extend the total number of epochs (otherwise it won't run)
n_epochs_new = 20

sim2.params.initial_epoch=sim.history.epoch[-1]+1 if len(sim.history.epoch)>0 else 0
sim2.params.totEpochs=sim2.params.initial_epoch + n_epochs_new

# We do not want the Train() method to load weights from somewhere, 
# so we make sure that load_weights is set to None
# If we wanted to load the weights from somewhere, 
# it would be enough to give it the right file name (usually, 'bestweights.hdf5')
sim2.UpdateParams(load_weights = None)

In [21]:
print('Epochs before running:', sim.history.epoch)
sim2.Train()
print('Epochs after running:', sim2.history.epoch)


Epochs before running: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
modelkind: image
Epoch 11/30
3/3 [==============================] - 0s 141ms/step - loss: 1.1652 - accuracy: 0.3820 - val_loss: 1.0905 - val_accuracy: 0.5122
Epoch 12/30
3/3 [==============================] - 0s 89ms/step - loss: 0.9433 - accuracy: 0.5393 - val_loss: 1.0811 - val_accuracy: 0.6585
Epoch 13/30
3/3 [==============================] - 0s 105ms/step - loss: 0.8141 - accuracy: 0.6458 - val_loss: 1.0755 - val_accuracy: 0.6829
Epoch 14/30
3/3 [==============================] - 0s 89ms/step - loss: 0.9037 - accuracy: 0.5843 - val_loss: 1.0706 - val_accuracy: 0.6829
Epoch 15/30
3/3 [==============================] - 0s 86ms/step - loss: 0.8458 - accuracy: 0.6067 - val_loss: 1.0665 - val_accuracy: 0.7073
Epoch 16/30
3/3 [==============================] - 0s 93ms/step - loss: 0.7851 - accuracy: 0.6458 - val_loss: 1.0627 - val_accuracy: 0.7073
Epoch 17/30
3/3 [==============================] - 0s 101ms/step - loss: 0.7515 - accur

# Use second model to predict on new, unseen data
Choose a class from the ones the classifier can recognize, and see whether the model is able to recognize it when taking unseen images in the validation dataset.

In [22]:

# We only do this for 'image' models, because the format of the validation directory is wrong and I don't want to cope with that now
if ttkind == 'image':
    target = sim2.params.class_select[1]
    print('target:',target)

    testdir = 'data/1_zooplankton_0p5x/validation/tommy_validation/images/'+target
    im_names=np.array(glob.glob(testdir+'/*.jpeg'),dtype=object)
    npimages=hd.LoadImageList(im_names, L=sim2.params.L, show=False)

    probs=sim2.model.predict(npimages)
    predictions=probs.argmax(axis=1)  # The class that the classifier would bet on

    print('Predictions:',sim2.tt.lb.classes_[predictions])

target: bosmina
Predictions: ['unknown_plankton' 'bosmina' 'bosmina' 'unknown_plankton' 'bosmina'
 'bosmina' 'bosmina' 'bosmina' 'bosmina']
